In [ ]:
import os
import pyrootutils

root = pyrootutils.setup_root(
    search_from=os.getcwd(),
    indicator=[".git", "pyproject.toml"],
    pythonpath=True,
    dotenv=True,
)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
#%matplotlib notebook
from omegaconf import OmegaConf

import pandas as pd

import tqdm
import glob
import ast
import re
import shutil
import numpy as np
import seaborn as sns
import mlflow
import matplotlib
import itertools

# Inspect Iterative Experiment Results
This notebook aids in inspecting the results of the iterative data chunks experiments, and in collecting the necessary information for the paper: gathering the figures and outputting latex table data.

In [ ]:
def get_model_name(row):
    if row["tags.ensemble"] == "True":
        return "ReWTS"
    if row["params.experiment/scale_model_parameters"] == "True":# and row["start_time"].year >= 2024:  #TODO: temp fix for xgb
        return "Global"
    else:
        return "Global No Scaling"


def search_mlflow(search_experiment_name, mlflow_dir=os.path.join(root, "logs", "mlflow", "mlruns")):
    tags_model_to_name = dict(XGB="XGBoost", TCN="TCN", RNN="LSTM", Regression="ElasticNet")
    if isinstance(search_experiment_name, str):
        search_experiment_name = [search_experiment_name]
    mlflow.set_tracking_uri(f"file://{mlflow_dir}")
    df = mlflow.search_runs(experiment_names=search_experiment_name)
    df['model_name'] = df.apply(get_model_name, axis=1)
    df['tags.model'] = df['tags.model'].apply(lambda x: tags_model_to_name[x.replace("Model", "")])

    return df
    

def set_matplotlib_attributes(font_size=8, font="DejaVu Sans"):
    matplotlib.rcParams.update({'font.size': font_size, 'font.family': font})

def set_figure_size(fig, column_span, height=None):
    if height is None:
        height = 5 if column_span == "double" else 10
    
    cm = 1 / 2.54
    if column_span == "single":
        fig_width = 8.4 * cm
    elif column_span == "double":
        fig_width = 17.4 * cm
    elif isinstance(column_span, (int, float)):
        fig_width = column_span
    else:
        raise ValueError()
    figsize = (fig_width, height * cm)

    fig.set_size_inches(*figsize)

def save_figure(fig, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    fig.savefig(path + ".pdf", format="pdf", bbox_inches="tight")
    fig.savefig(path + ".png", format="png", bbox_inches="tight")

## Performance Metrics

In [ ]:
fig_column_span = 10 #"double"
fig_height = 10
set_matplotlib_attributes(font_size=10)

tags_model_to_name = dict(XGB="XGBoost", TCN="TCN", RNN="LSTM", Regression="ElasticNet")
# Column names
chunk_idx_column = "params.datamodule/chunk_idx"
model_name_column = "model_name"
chunk_idx_plot_name = "Chunk #"

metric_name = "test_mse"
metric_plot_name = " ".join(metric_name.replace("test_", "").split("_")).upper()
metric_column = f"metrics.{metric_name}"

models = ["elastic-net-hopt"] #["xgboost", "elastic_net", "tcn", "rnn"]
dataset = "electricity"

broken = False
broken_limit = 100


for model in models:
    search_experiment_name = f"{dataset}_eval-it_{model}"
    search_experiment_name = [search_experiment_name, search_experiment_name + "-no-scale"]
    df = search_mlflow(search_experiment_name)

    if "Global No Scaling" in df["model_name"].values:
        model_order = ["ReWTS", "Global", "Global No Scaling"]
    else:
        model_order = ["ReWTS", "Global"]
    
    # Group DataFrame by 'chunk_idx' and 'model_name' and get the mean of the metric column
    grouped = df.groupby([chunk_idx_column, model_name_column])[metric_column].mean().reset_index()
    
    # Rename columns for better plotting
    grouped = grouped.rename(columns={metric_column: metric_plot_name, chunk_idx_column: chunk_idx_plot_name})
    
    # Sort by 'chunk_idx' numerically
    grouped[chunk_idx_plot_name] = grouped[chunk_idx_plot_name].astype(int)
    grouped[chunk_idx_plot_name] += 1
    #grouped = grouped[grouped[chunk_idx_plot_name] <= chunk_end]
    grouped = grouped.sort_values(by=[chunk_idx_plot_name, model_name_column])

    if broken:
        fig, (ax_higher, ax_lower) = plt.subplots(ncols=1, nrows=2, sharex=True)
        broken_lower = grouped.copy()
        broken_higher = grouped.copy()
    
        # Update the metric_plot_name column where the condition is met
        broken_lower.loc[broken_lower[metric_plot_name] >= broken_limit, metric_plot_name] = broken_limit
        broken_higher.loc[broken_higher[metric_plot_name] < broken_limit, metric_plot_name] = 0
    
        # Plotting
        catplot = sns.barplot(data=broken_lower, x=chunk_idx_plot_name, y=metric_plot_name, hue=model_name_column, ax=ax_lower)
        catplot = sns.barplot(data=broken_higher, x=chunk_idx_plot_name, y=metric_plot_name, hue=model_name_column, ax=ax_higher)

        ax_higher.set_ylim(bottom=broken_limit)
        ax_lower.set_ylim(0, broken_limit)

        set_figure_size(fig, fig_column_span)
    
        # Get current x-axis tick labels
        xticks = ax_lower.get_xticklabels()
        
        # Set every other tick label to be empty
        new_xticks = [xticks[i] if i % 2 == 0 else '' for i in range(len(xticks))]
        ax_lower.set_xticklabels(new_xticks)#, rotation=45)

        # the upper part does not need its own x axis as it shares one with the lower part
        ax_higher.get_xaxis().set_visible(False)

        higher_yticks = ax_higher.get_yticks()
        higher_yticks[0] = broken_limit
        ax_higher.set_yticks(higher_yticks)
        ax_higher.set_yticklabels(higher_yticks)
        
        # by default, each part will get its own "Latency in ms" label, but we want to set a common for the whole figure
        # first, remove the y label for both subplots
        ax_lower.set_ylabel("")
        ax_higher.set_ylabel("")

        #ax_higher.grid(visible=True, axes="y"
        # then, set a new label on the plot (basically just a piece of text) and move it to where it makes sense (requires trial and error)
        fig.text(0.05, 0.55, metric_plot_name, va="center", rotation="vertical")
    
        #catplot.set(ylim=(0,65))
    
        # Remove the legend title
        #new_legend = catplot.get_figure().get_legend()
        #new_legend.set_title('')
        # Set the legend inside the plot in the top left corner
        #new_legend.set_bbox_to_anchor((0.32, 0.90), transform=catplot.ax.transAxes) 
        # Remove the legend title
        legend = catplot.legend_
        legend.set_title('')
        legend.set_frame_on(False)
        ax_lower.get_legend().remove()
    else:
        # Plotting
        catplot = sns.catplot(data=grouped, x=chunk_idx_plot_name, y=metric_plot_name, hue=model_name_column, kind="bar", hue_order=model_order)
        fig = catplot.fig
        set_figure_size(fig, fig_column_span, height=fig_height)
    
        # Get current x-axis tick labels
        xticks = catplot.ax.get_xticklabels()
        
        # Set every other tick label to be empty
        new_xticks = [xticks[i] if i % 2 == 0 else '' for i in range(len(xticks))]
        catplot.set_xticklabels(new_xticks)#, rotation=45)
    
        #catplot.set(ylim=(0,65))
    
        # Remove the legend title
        new_legend = catplot._legend
    
        # Set the legend inside the plot in the top left corner
        new_legend.set_bbox_to_anchor((0.32, 0.90), transform=catplot.ax.transAxes)         
        new_legend.set_title('')

    # Set title
    plot_title = df["tags.model"][0]
    catplot.set(title=plot_title)

    fig_folder_name = search_experiment_name if isinstance(search_experiment_name, str) else search_experiment_name[0]


    fig_folder = os.path.join(root, "figures", fig_folder_name)
    save_figure(fig, os.path.join(fig_folder, f"chunk_{metric_name}_iterative_{dataset}_{model}"))
    grouped.to_csv(os.path.join(fig_folder, "iterative_dataframe.csv"))


In [ ]:
desired_decimals = 2
tags_model_to_name = dict(XGB="XGBoost", TCN="TCN", RNN="LSTM", Regression="ElasticNet")
print(df["tags.model"][0].replace("Model", ""))
#print(tags_model_to_name[df["tags.model"][0].replace("Model", "")])
mean_values = grouped.groupby("model_name")[metric_plot_name].mean()
# Properly set the float format option using a lambda function
pd.set_option('display.float_format', lambda x: f'{x:.{desired_decimals}e}')
mean_values

In [ ]:
def percentage_difference(value1, value2):
    return (value1 - value2) / ((value1 + value2) / 2) * 100

model_names = mean_values.index.unique()
combinations = itertools.combinations(model_names, 2)

for combo in combinations:
    percent_diff = percentage_difference(mean_values.loc[combo[0]], mean_values.loc[combo[1]])
    print(f"Percentage difference for {df['tags.model'][0].replace('Model', '')} between {combo[0]} and {combo[1]}: {percent_diff:.1f}%")

## Execution time

In [ ]:
chunk_end = 46
fig_column_span = "double"
set_matplotlib_attributes()

dataset = "electricity"
stages = ["train", "eval"]
stages_plot_name = ["Training", "Inference"]

tags_model_to_name = dict(XGB="XGBoost", TCN="TCN", RNN="LSTM", Regression="ElasticNet")

# Column names
chunk_idx_column = "params.experiment/chunk_idx"
model_name_column = "model_name"
chunk_idx_plot_name = "Chunk #"


for stage_i, stage in enumerate(stages):
    if stage == "eval":
        search_experiment_name = [f"{dataset}-eval_it-xgb-eval_time"]
    else:
        search_experiment_name = [f"{dataset}-train_time-xgboost-full", f"{dataset}-train_time-xgboost-ensemble"]
    df = search_mlflow(search_experiment_name)
    
    metric_name = f"{stage}_execution_time"
    metric_plot_name = f"{stages_plot_name[stage_i]} execution time (s)"
    metric_column = f"metrics.{metric_name}"

    if stage == "Inference":
        chunk_idx_column = "params.datamodule/chunk_idx"


    # Group DataFrame by 'chunk_idx' and 'model_name' and get the mean of the metric column
    grouped = df.groupby([chunk_idx_column, model_name_column])[metric_column].mean().reset_index()
    
    # Rename columns for better plotting
    grouped = grouped.rename(columns={metric_column: metric_plot_name, chunk_idx_column: chunk_idx_plot_name})
    
    # Sort by 'chunk_idx' numerically
    grouped[chunk_idx_plot_name] = grouped[chunk_idx_plot_name].astype(int)
    grouped[chunk_idx_plot_name] += 1
    grouped = grouped[grouped[chunk_idx_plot_name] <= chunk_end]
    grouped = grouped.sort_values(by=[chunk_idx_plot_name, model_name_column])
    
    if stage == "train":
        # Accumulate the metric
        grouped[metric_plot_name] = grouped.groupby(model_name_column)[metric_plot_name].cumsum()
    
    # Plotting
    catplot = sns.catplot(data=grouped, x=chunk_idx_plot_name, y=metric_plot_name, hue=model_name_column, kind="bar")
    set_figure_size(catplot.fig, column_span=fig_column_span)
    
    plt.yscale('log')
    # Rotate x-axis labels for better readability
    catplot.set_xticklabels(rotation=45)
    
    #catplot.set(ylim=(0,65))
    
    # Remove the legend title
    new_legend = catplot._legend
    new_legend.set_title('')
    # Set the legend inside the plot in the top left corner
    new_legend.set_bbox_to_anchor((0.25, 0.95), transform=catplot.ax.transAxes) 
    
    # Set title
    plot_title = df["tags.model"][0]
    catplot.set(title=plot_title)
    
    fig_folder_name = f"execution_time_{dataset}"
    fig_path = os.path.join(root, "figures", fig_folder_name, f"execution_time_{stages_plot_name[stage_i]}")
    save_figure(catplot.figure, fig_path)
    plt.show()